# Heuristics - CVRP

[OR-Tools Documentation](https://or-tools.github.io/docs/pdoc/ortools/constraint_solver/pywrapcp.html)

To do:

- Implement skills
- Allow incomplete jobs

In [1]:
from itertools import cycle

import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import matplotlib as mpl
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import functions as f
import importlib

## Instantiate data

In [2]:
n_vehicles = 5
seniority_multiplier = [1, 1, 1.2, 1.6, 1]
start_locations = [4,5,6,7,8]
end_locations = [0,0,0,0,0]

dataset = pd.read_csv("./data/custom_19.csv", index_col=0)
dataset_copy = dataset.copy()

for home_indx in start_locations:
    dataset.loc[home_indx, "timeslot"] = "AD"
    dataset.loc[home_indx, "job_type"] = "start"
    
coordinates = dataset.loc[:, ["x", "y"]]

N = coordinates.shape[0]

In [3]:
dataset

,x,y,timeslot,job_type
0,0,0,AD,end
1,-14,12,PM,DF-M Commision
2,-84,11,PM,DF-M Commision
3,-60,5,AM,DF-M Commision
4,79,64,AD,start
5,11,87,AD,start
6,8,68,AD,start
7,-4,24,AD,start
8,-11,20,AD,start
9,22,-5,AM,DF-M Exchange


In [4]:
# time windows, each each number represents 1 minute
time_window_dict = {
    "AD": [0, 510],  # 08:00 - 16:30
    "AM": [0, 240],  # 08:00 - 12:00
    "PM": [240, 510],  # 12:00 - 16:30
}

service_time_dict = {
    "start": 0,
    "DF-M Commision": 50,
    "DF-M Exchange": 100,
    "end": 0
}

In [5]:
time_matrix_multiplier = 1  # used to make the drive times fill the day more

# assume travel time is proportional to euclidean distance
time_matrix = squareform(pdist(coordinates, metric="euclidean"))

# reduce the drive times to make feasible solutions more likely, will adjust later!
time_matrix = np.round(time_matrix * time_matrix_multiplier, decimals=0).astype(int)

time_matrix[:,0] = time_matrix[0,:] = 0 # the end location is a dummy variable used to allow arbitrary end locations

## Model

In [6]:
# Create the routing index manager: number of nodes, number of vehicles, depot node
manager = pywrapcp.RoutingIndexManager(N, n_vehicles, start_locations, end_locations)

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

In [7]:
dataset

,x,y,timeslot,job_type
0,0,0,AD,end
1,-14,12,PM,DF-M Commision
2,-84,11,PM,DF-M Commision
3,-60,5,AM,DF-M Commision
4,79,64,AD,start
5,11,87,AD,start
6,8,68,AD,start
7,-4,24,AD,start
8,-11,20,AD,start
9,22,-5,AM,DF-M Exchange


### Parameters

In [8]:
# must create a different time_callback for
def create_time_callback(vehicle_id):
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        travel_time = time_matrix[from_node, to_node]

        # convention is that job time added to drive FROM the job's node
        service_time = int(
            service_time_dict[dataset.loc[from_node, "job_type"]]
            * seniority_multiplier[vehicle_id]
        )

        return travel_time + service_time

    return time_callback


transit_callback_indices = []

for vehicle_id in range(n_vehicles):
    transit_callback = create_time_callback(vehicle_id)
    transit_callback_index = routing.RegisterTransitCallback(transit_callback)
    transit_callback_indices.append(transit_callback_index)

### Constraints

In [9]:
# Define time window contraints
time = "Time"

# create a CumulVar, for each location AND each vehicle
# 0 to N_locations-1, N_locations to N_locations+N_vehicles-1
routing.AddDimensionWithVehicleTransits(
    transit_callback_indices,
    600,  # allow waiting time
    600,  # maximum time per vehicle (and location but the range is set later)
    True,  # start cumul to zero
    time,
)

time_dimension = routing.GetDimensionOrDie(time)

for location_indx in dataset.index:
    timeslot = dataset.loc[location_indx, "timeslot"]

    time_dimension.CumulVar(location_indx).SetRange(
        time_window_dict[timeslot][0],
        time_window_dict[timeslot][1],  # window depends on promised timeslot
    )

### Objective

In [10]:
# The cost of each arc is vehicle dependant!
for vehicle_id in range(n_vehicles):
    routing.SetArcCostEvaluatorOfVehicle(
        transit_callback_indices[vehicle_id], vehicle_id
    )

### Solution

In [11]:
# Setting heuristic strategies
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)

search_parameters.time_limit.FromSeconds(5)

# Solve the problem
solution = routing.SolveWithParameters(search_parameters)

In [12]:
solution.ObjectiveValue()

1381

### Plot Results

In [13]:
calculated_objective_function_value = 0

print_val = "Key: |Node| A:Arrival, JT:Job Time, T:Travel Time, J:Job Time\n"
for vehicle_id in range(n_vehicles):
    index = routing.Start(vehicle_id)
    previous_index = None

    total_time = 0
    while not routing.IsEnd(index):

        time_var = time_dimension.CumulVar(index)

        arrival_time = solution.Value(time_var)

        previous_index = index
        previous_node = manager.IndexToNode(index)

        index = solution.Value(routing.NextVar(index))
        node = manager.IndexToNode(index)

        job_drive_time = routing.GetArcCostForVehicle(previous_index,index,vehicle_id)

        total_time += job_drive_time

        print_val += f"|{previous_node}| A:{arrival_time}, TS: {dataset.loc[previous_node, 'timeslot']}, JT:{job_drive_time}, T:{time_matrix[previous_node][node]}, J:{service_time_dict[dataset.loc[previous_node, 'job_type']]} -> "

    index = routing.End(vehicle_id)
    print_val += f"{total_time} min\n"

    vehicle_objective = solution.Value(time_dimension.CumulVar(index))
    print_val += f"Vehicle Objective: {vehicle_objective} min \n \n"

    calculated_objective_function_value += vehicle_objective

print(print_val)

print(f"Sum of above: {calculated_objective_function_value} min")
print(f"Objective function value: {solution.ObjectiveValue()} min")



Key: |Node| A:Arrival, JT:Job Time, T:Travel Time, J:Job Time
|4| A:0, TS: AD, JT:35, T:35, J:0 -> |13| A:35, TS: PM, JT:105, T:55, J:50 -> |12| A:140, TS: AM, JT:100, T:0, J:100 -> 240 min
Vehicle Objective: 240 min 
 
|5| A:0, TS: AD, JT:62, T:62, J:0 -> |18| A:62, TS: AM, JT:133, T:33, J:100 -> |10| A:195, TS: AD, JT:56, T:6, J:50 -> |9| A:251, TS: AM, JT:216, T:116, J:100 -> |17| A:467, TS: AD, JT:50, T:0, J:50 -> 517 min
Vehicle Objective: 517 min 
 
|6| A:0, TS: AD, JT:46, T:46, J:0 -> |16| A:46, TS: AD, JT:60, T:0, J:50 -> 106 min
Vehicle Objective: 106 min 
 
|7| A:0, TS: AD, JT:0, T:0, J:0 -> 0 min
Vehicle Objective: 0 min 
 
|8| A:0, TS: AD, JT:9, T:9, J:0 -> |1| A:9, TS: PM, JT:138, T:88, J:50 -> |11| A:147, TS: AD, JT:75, T:25, J:50 -> |15| A:222, TS: AM, JT:101, T:1, J:100 -> |14| A:323, TS: AM, JT:70, T:20, J:50 -> |2| A:393, TS: PM, JT:75, T:25, J:50 -> |3| A:468, TS: AM, JT:50, T:0, J:50 -> 518 min
Vehicle Objective: 518 min 
 

Sum of above: 1381 min
Objective function

In [20]:
for location_indx in dataset.index:
    timeslot = dataset.loc[location_indx, "timeslot"]
    time_var = time_dimension.CumulVar(location_indx)
    print(f"Location {location_indx} ({timeslot}): {solution.Min(time_var)} - {solution.Max(time_var)}, {solution.Value(time_var)}")

Location 0 (AD): 9 - 9, 9
Location 1 (PM): 393 - 393, 393
Location 2 (PM): 468 - 468, 468
Location 3 (AM): 0 - 0, 0
Location 4 (AD): 0 - 0, 0
Location 5 (AD): 0 - 0, 0
Location 6 (AD): 0 - 0, 0
Location 7 (AD): 0 - 0, 0
Location 8 (AD): 251 - 294, 251
Location 9 (AM): 195 - 238, 195
Location 10 (AD): 147 - 147, 147
Location 11 (AD): 140 - 140, 140
Location 12 (AM): 35 - 35, 35
Location 13 (PM): 323 - 323, 323
Location 14 (AM): 222 - 222, 222
Location 15 (AM): 46 - 240, 46
Location 16 (AD): 467 - 510, 467
Location 17 (AD): 62 - 105, 62
Location 18 (AM): 240 - 240, 240


In [15]:
dataset

,x,y,timeslot,job_type
0,0,0,AD,end
1,-14,12,PM,DF-M Commision
2,-84,11,PM,DF-M Commision
3,-60,5,AM,DF-M Commision
4,79,64,AD,start
5,11,87,AD,start
6,8,68,AD,start
7,-4,24,AD,start
8,-11,20,AD,start
9,22,-5,AM,DF-M Exchange


In [16]:
# total_time = 0
# for index in range(N, N + n_vehicles):
#     time_var = time_dimension.CumulVar(index)
#     vehicle_time = solution.Min(time_var)
#     total_time += vehicle_time
#     print(
#         "Vehicle {} has a total time of {}min".format(
#             index - N, vehicle_time
#         )
#     )
# print(f"Total time for all vehicles: {total_time}min")
# print(f"Objective value: {solution.ObjectiveValue()}")

In [17]:
importlib.reload(f)
tours = f.compile_tours(
    n_vehicles, routing, manager, solution, True
)

In [18]:
importlib.reload(f)
fig = f.plot_tours(tours, coordinates)
fig.show()

In [ ]:
importlib.reload(f)
fig = f.plot_tours(tours, coordinates)
fig.show()